# Отчет по домашней работе №4: Вопросно-ответные системы


## Импорты и подпрограммы

In [7]:
import os
import json
import torch
import gensim
import numpy as np
import pandas as pd
import lightgbm as lgb
from itertools import islice
import matplotlib.pyplot as plt
from transformers import AutoModel
from transformers import AutoTokenizer
from sklearn.metrics import (
    roc_auc_score, 
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score
)


PATH_TRAIN = "../data/train.jsonl"
PATH_TEST = "../data/dev.jsonl"


def log(msg:str, headers=None):
    dttm = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    if (
        ("__main__" in globals() or "__main__" in locals())
        and hasattr(__main__, "__file__")
    ):
        script = os.path.basename(__main__.__file__)
    else:
        script = "jupyter"
    if headers is None:
        headers = []
    header_line = f"[{dttm}][{script}]" + "".join(f"[{h}]" for h in headers)
    print(f"{header_line} {msg}")


def batched(iterable, n):
    # batched('ABCDEFG', 3) → ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while batch := tuple(islice(it, n)):
        yield batch


def read_data(path):
    records = []
    with open(path) as fp:
        for line in fp:
            record = json.loads(line.strip())
            records.append(record)
    df = pd.DataFrame(records)
    return df


def prepare(df, batch_size=128):
    df["tp"] = df.question + ". " + df.passage
    texts = df.tp.tolist()
    batches = batched(texts, batch_size)
    return batches
    

def make_bert_embs(batches):
    b_vecs = []
    for idx, batch in enumerate(batches):
        if idx%5==0:
            log(f"{idx} batches complete", ["make_embs"])
        tokens = tokenizer(
            batch, 
            return_tensors="pt", 
            padding=True, 
            truncation=True,
            max_length=512
        )
        with torch.no_grad():
            vecs = (
                model(**tokens)
                .last_hidden_state
                .mean(1)
                .detach()
                .numpy()
            )
        b_vecs.append(vecs)
    return np.vstack(b_vecs)
    

def get_train_bert_embs():
    return get_bert_embs("train.npy", PATH_TRAIN)


def get_test_bert_embs():
    return get_bert_embs("test.npy", PATH_TEST)


def get_bert_embs(filename, path_in, cache_dir="./"):
    fpath = os.path.join(cache_dir, filename)
    if os.path.exists(fpath):
        with open(fpath, 'rb') as fp:
            embs = np.load(fp)
    else:
        batches = prepare(read_data(path_in))  # 590
        embs = make_bert_embs(batches)
        with open(fpath, 'wb') as fp:
            np.save(fp, embs)
    return embs


def get_scores(y_true, y_score, thr=0.5):
    y_pred = (thr<y_score).astype(int)
    auc = roc_auc_score(y_score=y_score, y_true=y_true)
    f1 = f1_score(y_true=y_true, y_pred=y_pred)
    prec = precision_score(y_true=y_true, y_pred=y_pred)
    rec = recall_score(y_true=y_true, y_pred=y_pred)
    acc = accuracy_score(y_true=y_true, y_pred=y_pred)
    return {
        "auroc": auc,
        "f1": f1,
        "precision": prec,
        "recall": rec,
        "accuracy": acc
    }


def print_scores(msg, rec):
    scores = ", ".join(
        f"{k}={v}" for k,v in rec.items()
    )
    print(f"{msg}: {scores}")



## Часть 1. [1 балл] Эксплоративный анализ
1. Посчитайте долю yes и no классов в корпусе
2. Оцените среднюю длину вопроса
3. Оцените среднюю длину параграфа
4. Предположите, по каким эвристикам были собраны вопросы (или найдите ответ в статье). Продемонстриуйте, как эти эвристики повлияли на структуру корпуса. 

In [2]:
d_train = read_data(PATH_TRAIN)
d_test = read_data(PATH_TEST)

### 1.1 Оценка средней доли YES в корпусе
Доля NO это 1-YES

In [3]:
pd.DataFrame(
    [
        {
            "scope": "train", 
            "mean": d_train.answer.astype(int).mean(),
            "std": d_train.answer.astype(int).std(),
        },
        {
            "scope": "test", 
            "mean": d_test.answer.astype(int).mean(),
            "std": d_test.answer.astype(int).std(),
        }
    ]
)


,scope,mean,std
0,train,0.623104,0.484634
1,test,0.621713,0.485034


### 1.2 Оценка средней длины вопроса


In [4]:
pd.DataFrame(
    [
        {
            "scope": "train", 
            "mean": d_train.question.str.len().mean(),
            "std": d_train.question.str.len().std(),
        },
        {
            "scope": "test", 
            "mean": d_test.question.str.len().mean(),
            "std": d_test.question.str.len().std(),
        }
    ]
)

,scope,mean,std
0,train,43.991938,8.854335
1,test,43.206422,7.785706


### 1.3 Оценка средней длины параграфа


In [5]:
pd.DataFrame(
    [
        {
            "scope": "train", 
            "mean": d_train.passage.str.len().mean(),
            "std": d_train.passage.str.len().std(),
        },
        {
            "scope": "test", 
            "mean": d_test.passage.str.len().mean(),
            "std": d_test.passage.str.len().std(),
        }
    ]
)

,scope,mean,std
0,train,565.613026,323.137498
1,test,559.052294,328.796047


### 1.4 Структура корпуса
Датасет собирался на основе датасета Natural Questions Corpus, который состоит из пар <Поисковый запрос, текст с ответом>. 

Датасет имеетследующие особенности:
- Выбирались такие пары, где запрос начинался на какое-либо из следующих слов: “did”, “do”, “does”, “is”, “are”, “was”, “were”, “have”, “has”, “can”, “could”, “will”, “would”.
- Также отсекалсь слишом короткие вопросы.
- В датасете ответ на запрос всегда можно локализовать в тексте. То есть нет вопросов, требующих вывод по тексту вместо поиска по тексту.
- Аннотаторы из-всего текста выбирали конкретный параграф, в котором содержался ответ. Поэтому текст всегда представляет собой один параграф.



## Часть 2. [1 балл] Baseline
1. Оцените accuracy точность совсем простого базового решения: присвоить каждой паре вопрос-ответ в dev части самый частый класс из train части
2. Оцените accuracy чуть более сложного базового решения: fasttext на текстах, состоящих из склееных вопросов и абзацев (' '.join([question, passage]))

Почему fasttext плохо справляется с этой задачей?

# Бейзлайн 1: Наиболее частый ответ

In [6]:
d_train = read_data(PATH_TRAIN)
d_test = read_data(PATH_TEST)


In [32]:
ans_train = d_train.answer.mode().values.item()
# ans_test = d_test.answer.mode().values.item()
p_train = np.array([ans_train for i in range(d_train.shape[0])])
p_test = np.array([ans_train for i in range(d_test.shape[0])])
print(f"Наиболее частый ответ: {ans_train=}")
bl1_train = get_scores(y_true=d_train.answer.tolist(), y_score=p_train)
bl1_test = get_scores(y_true=d_test.answer.tolist(), y_score=p_test)
print("Метрики Бейзлайн-1 на обучающей выборке", bl1_train)
print("Метрики Бейзлайн-1 на тестовой выборке", bl1_test)


Наиболее частый ответ: ans_train=True
Метрики Бейзлайн-1 на обучающей выборке {'auroc': 0.5, 'f1': 0.7677929547088426, 'precision': 0.6231038506417736, 'recall': 1.0, 'accuracy': 0.6231038506417736}
Метрики Бейзлайн-1 на тестовой выборке {'auroc': 0.5, 'f1': 0.7667358099189139, 'precision': 0.6217125382262997, 'recall': 1.0, 'accuracy': 0.6217125382262997}


# Бейзлайн 2: FastText

In [17]:
model = gensim.models.fasttext.load_facebook_vectors("../data/cc.ru.300.bin")
d_train["tp"] = d_train.question + ". " + d_train.passage
d_test["tp"] = d_test.question + ". " + d_test.passage
tokens_train = d_train.tp.apply(lambda line: [x for x in gensim.utils.tokenize(line)]).tolist()
tokens_test = d_test.tp.apply(lambda line: [x for x in gensim.utils.tokenize(line)]).tolist()
x_train = np.array(
    [model.get_mean_vector(tokens).tolist() for tokens in tokens_train]
)
x_test =  np.array(
    [model.get_mean_vector(tokens).tolist() for tokens in tokens_test]
)
# params = dict(min_child_samples=10) # 0.7706774951912803
params = dict(min_child_samples=8) # 0.7706774951912803
est = lgb.LGBMClassifier(
    **params
)
est.fit(x_train, d_train.answer.values)
bl2_train = get_scores(
    y_true=d_train.answer.values,
    y_score=est.predict_proba(x_train)[:,1]
)
print("Метрики Бейзлайн-2(FastText) на обучающей выборке: ", bl2_train)
bl2_test = get_scores(
    y_true=d_test.answer.values,
    y_score=est.predict_proba(x_test)[:,1]
)
print("Метрики Бейзлайн-2(FastText) на тестовой выборке: ", bl2_test)


[LightGBM] [Info] Number of positive: 5874, number of negative: 3553
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012624 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 9427, number of used features: 300
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.623104 -> initscore=0.502744
[LightGBM] [Info] Start training from score 0.502744
Метрики Бейзлайн-2(FastText) на обучающей выборке:  {'auroc': 0.9948531939277219, 'f1': 0.9497360941940722, 'precision': 0.9079335506908865, 'recall': 0.9955737146748382, 'accuracy': 0.9343375411053357}
Метрики Бейзлайн-2(FastText) на тестовой выборке:  {'auroc': 0.6582806490004656, 'f1': 0.7660297239915074, 'precision': 0.673888681359731, 'recall': 0.8873585833743236, 'accuracy': 0.6629969418960244}


# Часть 3. [1 балл] Используем эмбеддинги предложений

1. Постройте BERT эмбеддинги вопроса и абзаца. Обучите логистическую регрессию на конкатенированных эмбеддингах вопроса и абзаца и оцените accuracy этого решения. 

[bonus] Используйте другие модели эмбеддингов, доступные, например, в библиотеке 🤗 Transformers. Какая модель эмбеддингов даст лучшие результаты?

[bonus] Предложите метод аугментации данных и продемонстрируйте его эффективность. 


In [22]:
# считаем ruBert-эмбеддинги или загружаем из кэша
x_train = get_train_bert_embs()
x_test = get_test_bert_embs()
d_train = read_data(PATH_TRAIN)
d_test = read_data(PATH_TEST)
y_train = d_train.answer.astype(int)
y_test = d_test.answer.astype(int)


# params = dict(min_child_samples=8) # 0.669559980610946
params = dict(min_child_samples=100) # 0.669559980610946

est = lgb.LGBMClassifier(**params)
est.fit(x_train, d_train.answer.values)

bert_train = get_scores(
    y_true=d_train.answer.values,
    y_score=est.predict_proba(x_train)[:,1]
)
bert_test = get_scores(
    y_true=d_test.answer.values,
    y_score=est.predict_proba(x_test)[:,1]
)


print(f"Метрики с Bert-эмбеддингами на обучающей выборке: {bert_train}")
print(f"Метрики с Bert-эмбеддингами на тестовой выборке: {bert_test}")

[LightGBM] [Info] Number of positive: 5874, number of negative: 3553
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.033508 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 9427, number of used features: 768
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.623104 -> initscore=0.502744
[LightGBM] [Info] Start training from score 0.502744
Метрики с Bert-эмбеддингами на обучающей выборке: {'auroc': 0.9957578517475676, 'f1': 0.9628714909031036, 'precision': 0.9322493224932249, 'recall': 0.9955737146748382, 'accuracy': 0.9521586931155193}
Метрики с Bert-эмбеддингами на тестовой выборке: {'auroc': 0.6787914527515079, 'f1': 0.767217335335765, 'precision': 0.680365296803653, 'recall': 0.8794884407279882, 'accuracy': 0.6681957186544343}


# Часть 4. [3 балла] DrQA-подобная архитектура
<h1>...</h1>

# Часть 5. [3 балла] BiDAF-подобная архитектура
<h1>...</h1>

# Часть 6. [1 балл] Итоги

## 6.1. Сравнение результатов
Были рассмотрены такие вариаенты как: 
- Наиболее частый ответ (Бейзлайн-1)
- XGB + FastText-эмбеддинги (Бейзлайн-2)
- XGB + Bert-эмбеддинги

Для сравнения по f1, precision, recall и accuracy для простоты использовался порог равный 0.5, можно попробовать выбирать наилучший порог: максимизировать f1.

Сравним результаты обучения моделей

In [48]:
d_scores = pd.DataFrame(
    [
        {'model': 'bl1_mode', 'scope': 'train', 'auroc': 0.5, 'f1': 0.7677929547088426, 'precision': 0.6231038506417736, 'recall': 1.0, 'accuracy': 0.6231038506417736},
        {'model': 'bl1_mode', 'scope': 'test', 'auroc': 0.5, 'f1': 0.7667358099189139, 'precision': 0.6217125382262997, 'recall': 1.0, 'accuracy': 0.6217125382262997},
        
        {'model': 'fasttext', 'scope': 'train', 'auroc': 0.9948531939277219, 'f1': 0.9497360941940722, 'precision': 0.9079335506908865, 'recall': 0.9955737146748382, 'accuracy': 0.9343375411053357},
        {'model': 'fasttext', 'scope': 'test', 'auroc': 0.6582806490004656, 'f1': 0.7660297239915074, 'precision': 0.673888681359731, 'recall': 0.8873585833743236, 'accuracy': 0.6629969418960244},
        
        {'model': 'bert', 'scope': 'train', 'auroc': 0.9976606494140341, 'f1': 0.966922378949105, 'precision': 0.9379100656104977, 'recall': 0.9977868573374191, 'accuracy': 0.9574626074042644},
        {'model': 'bert', 'scope': 'test', 'auroc': 0.6787914527515079, 'f1': 0.767217335335765, 'precision': 0.680365296803653, 'recall': 0.8794884407279882, 'accuracy': 0.6681957186544343}
    ]
)

In [52]:
d_scores[d_scores.scope=="train"]

,model,scope,auroc,f1,precision,recall,accuracy
0,bl1_mode,train,0.500000,0.767793,0.623104,1.000000,0.623104
2,fasttext,train,0.994853,0.949736,0.907934,0.995574,0.934338
4,bert,train,0.997661,0.966922,0.937910,0.997787,0.957463


In [50]:
d_scores[d_scores.scope=="test"]

,model,scope,auroc,f1,precision,recall,accuracy
1,bl1_mode,test,0.500000,0.766736,0.621713,1.000000,0.621713
3,fasttext,test,0.658281,0.766030,0.673889,0.887359,0.662997
5,bert,test,0.678791,0.767217,0.680365,0.879488,0.668196


Наилучшие результаты по ROC AUC, F1, Accuracy у модели Bert+XGB. Затем модель FastText+XGB превосходит первый бейзлайн по ROC AUC и Accuracy.

# 6.2 Резюме о проделанной работе
Были выполнены такие модели как:
- Наиболее частый ответ (Бейзлайн-1)
- XGB + FastText-эмбеддинги (Бейзлайн-2)
- XGB + Bert-эмбеддинги

Не выполены модели:
- DrQA
- BiDAF

Надеюсь успеть сделать хотя бы одну из этих моделей. Выполнению моделей пока помешало что нет готовых имплементаций для BoolQ  моделей DrQA и BiDAF, надо переделывать модели под задание. Этого мне не хватает. А помогла библиотека HuggngFace - очень удобная.